In [61]:
import pandas as pd
import numpy as np

# Data Indexing and Selection

![](https://drive.google.com/uc?export=view&id=1hLo7Y7HRrh6Fd3zEgIlZtLnmOUH_ZRbk)<br>

---


In NumPy arrays there are multiple methods and tools to access, set, and modify values array values.
These include indexing (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``), masking (e.g., ``arr[arr > 0]``), fancy indexing (e.g., ``arr[0, [1, 5]]``), and combinations thereof (e.g., ``arr[:, [1, 5]]``).
Here we'll look at similar means of accessing and modifying values in Pandas ``Series`` and ``DataFrame`` objects.

We'll start with the simple case of the one-dimensional ``Series`` object, and then move on to the more complicated two-dimesnional ``DataFrame`` object.

## Data Selection in Series

As we saw in the previous section, a ``Series`` object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary.
If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as one-dimensional array

A ``Series`` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms � *slices*, *masking*, and *fancy indexing*.
Examples of these are as follows:

In [62]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [63]:
# slicing by explicit index
data['b':'d']

b    0.50
c    0.75
d    1.00
dtype: float64

In [64]:
# slicing by implicit integer index
data[1:3]

b    0.50
c    0.75
dtype: float64

In [65]:
# masking
data[(data > 0.1) & (data < 0.5)]

a    0.25
dtype: float64

In [66]:
# fancy indexing
data[['a','c','d']]

a    0.25
c    0.75
d    1.00
dtype: float64

In [67]:
data['a':'d']

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

Among these, slicing may be the source of the most confusion.
Notice that when slicing with an explicit index (i.e., ``data['a':'c']``), the final index is *included* in the slice, while when slicing with an implicit index (i.e., ``data[0:2]``), the final index is *excluded* from the slice.

### Indexers: loc, iloc

These slicing and indexing conventions can be a source of confusion.
For example, if your ``Series`` has an explicit integer index, an indexing operation such as ``data[1]`` will use the explicit indices, while a slicing operation like ``data[1:3]`` will use the implicit Python-style index.

In [68]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [69]:
# explicit index when indexing
data[1]

'a'

In [70]:
# implicit index when slicing
data[3:5]

Series([], dtype: object)

In [71]:
data[1:3]

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indexes, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes.
These are not functional methods, but attributes that expose a particular slicing interface to the data in the ``Series``.

First, the ``loc`` attribute allows indexing and slicing that always references the explicit index:

In [72]:
data

1    a
3    b
5    c
dtype: object

In [73]:
data.loc[1]

'a'

In [74]:
data.loc[1:3]

1    a
3    b
dtype: object

The ``iloc`` attribute allows indexing and slicing that always references the implicit Python-style index:

In [75]:
data.iloc[1]

'b'

In [76]:
data.iloc[1:3]

3    b
5    c
dtype: object

One guiding principle of Python code is that "explicit is better than implicit."
The explicit nature of ``loc`` and ``iloc`` make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

## Data Selection in DataFrame

A ``DataFrame`` acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of ``Series`` structures sharing the same index.
These analogies can be helpful to keep in mind as we explore data selection within this structure.

### DataFrame as a dictionary

The first analogy we will consider is the ``DataFrame`` as a dictionary of related ``Series`` objects.
Let's return to our example of areas and populations of states:

In [77]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


The individual ``Series`` that make up the columns of the ``DataFrame`` can be accessed via dictionary-style indexing of the column name:

In [78]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Equivalently, we can use attribute-style access with column names that are strings:

In [79]:
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [80]:
data.area is data['area']

True

Though this is a useful shorthand, keep in mind that it does not work for all cases!
For example, if the column names are not strings, or if the column names conflict with methods of the ``DataFrame``, this attribute-style access is not possible.
For example, the ``DataFrame`` has a ``pop()`` method, so ``data.pop`` will point to this rather than the ``"pop"`` column:

In [81]:
data.pop is data['pop']

False

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use ``data['pop'] = z`` rather than ``data.pop = z``).

Like with the ``Series`` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [82]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [83]:
data['abc'] = data['pop']*data['area']
data

,area,pop,density,abc
California,423967,38332521,90.413926,16251723930807
Texas,695662,26448193,38.018740,18399002838766
New York,141297,19651127,139.076746,2776645291719
Florida,170312,19552860,114.806121,3330086692320
Illinois,149995,12882135,85.883763,1932255839325


This shows a preview of the straightforward syntax of element-by-element arithmetic between ``Series`` objects.

### DataFrame as two-dimensional array

As mentioned previously, we can also view the ``DataFrame`` as an enhanced two-dimensional array.
We can examine the raw underlying data array using the ``values`` attribute:

In [84]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01, 1.62517239e+13],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01, 1.83990028e+13],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02, 2.77664529e+12],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02, 3.33008669e+12],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01, 1.93225584e+12]])

With this picture in mind, many familiar array-like observations can be done on the ``DataFrame`` itself.
For example, we can transpose the full ``DataFrame`` to swap rows and columns:

In [85]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01
abc,1.625172e+13,1.839900e+13,2.776645e+12,3.330087e+12,1.932256e+12


When it comes to indexing of ``DataFrame`` objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array.
In particular, passing a single index to an array accesses a row:

In [86]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01, 1.62517239e+13],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01, 1.83990028e+13],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02, 2.77664529e+12],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02, 3.33008669e+12],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01, 1.93225584e+12]])

In [87]:
data.values[1]

array([6.95662000e+05, 2.64481930e+07, 3.80187404e+01, 1.83990028e+13])

Thus for array-style indexing, we need another convention.
Here Pandas again uses the ``loc``and ``iloc`` indexers mentioned earlier.
Using the ``iloc`` indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the ``DataFrame`` index and column labels are maintained in the result:

In [88]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


Similarly, using the ``loc`` indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [89]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Any of the familiar NumPy-style data access patterns can be used within these indexers.
For example, in the ``loc`` indexer we can combine masking and fancy indexing as in the following:

In [90]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [91]:
data.iloc[0, 2] = 90
data

,area,pop,density,abc
California,423967,38332521,90.000000,16251723930807
Texas,695662,26448193,38.018740,18399002838766
New York,141297,19651127,139.076746,2776645291719
Florida,170312,19552860,114.806121,3330086692320
Illinois,149995,12882135,85.883763,1932255839325


### Index setting and resetting

In Pandas dataframe, you can set any column as an index for the dataframe. The column can replace the existing index or expand on it. You can use the new column index similar to how you would use a normal index.

In [92]:
population = [{'state':'California', 'year':2000, 'population':33871648}, 
         {'state':'California', 'year':2010, 'population':37253956},
         {'state':'New York', 'year':2000,'population':18976457}, 
         {'state':'New York', 'year':2010, 'population':19378102},
         {'state':'Texas', 'year':2000, 'population':20851820}, 
         {'state':'Texas', 'year':2010, 'population':25145561}]
pop_flat = pd.DataFrame(population)
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


You can use the `set_index` method to set any column as index for the Dataframe. Let us set the state column as index for the population dataframe.

In [93]:
pop_state = pop_flat.set_index('state')
pop_state

,year,population
state,,
California,2000,33871648
California,2010,37253956
New York,2000,18976457
New York,2010,19378102
Texas,2000,20851820
Texas,2010,25145561


Now, you can use `loc` to access the dataframe using the state names now. 

In [94]:
pop_state.loc['California']

,year,population
state,,
California,2000,33871648
California,2010,37253956


Note that `iloc` will work the same way as before.

In [95]:
pop_state.iloc[1]

year              2010
population    37253956
Name: California, dtype: int64

Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [96]:
pop = pop_flat.set_index(['state', 'year'])
pop

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

Now to access all data using both the state name and year index. In this way, all the values can be easily accessed.

In [97]:
pop.loc['California',2000]

population    33871648
Name: (California, 2000), dtype: int64

Selection based on fancy indexing also works:

In [98]:
pop.loc[['California','Texas']]

population
state      year            
California 2000    33871648
           2010    37253956
Texas      2000    20851820
           2010    25145561

You can use `reset_index` to reset the dataframe to a default index. In this case the existing index values will be reset to new columns in the Dataframe.

In [99]:
pop_flat = pop.reset_index()
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561
